In [ ]:
!pip install -q -U transformers datasets bitsandbytes  trl peft  huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# Prompt:       [PAD, PAD, 1, 2]       (left-pad)
# Chosen:       [3, 4, 5, PAD]         (right-pad)
# Rejected:     [6, PAD, PAD, PAD]     (right-pad)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch


name_name = "Qwen/Qwen2.5-1.5B-Instruct"

config_8bit = BitsAndBytesConfig(load_in_8bit=True)


model_8bit = AutoModelForCausalLM.from_pretrained(

                                                  name_name,
                                                  quantization_config=config_8bit,
                                                  trust_remote_code=True,
                                                  device_map="auto",
                                                  )

tokenizer = AutoTokenizer.from_pretrained(name_name, trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

dataset = load_dataset("HumanLLMs/Human-Like-DPO-Dataset",split="train[:300]")
dataset

README.md:   0%|          | 0.00/1.69k [00:00<?, ?B/s]

data.json:   0%|          | 0.00/28.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10884 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 300
})

In [ ]:
dataset[0]

{'prompt': 'Oh, I just saw the best meme - have you seen it?',
 'chosen': "😂 Ah, no I haven't! I'm dying to know, what's the meme about? Is it a funny cat or a ridiculous situation? Spill the beans! 🤣",
 'rejected': "I'm an artificial intelligence language model, I don't have personal experiences or opinions. However, I can provide you with information on highly-rated and critically acclaimed films, as well as recommendations based on specific genres or themes. Would you like me to suggest some notable movies or discuss a particular genre of interest?"}

In [ ]:
example_text = [{'role':'assistant','content':dataset[0]['chosen']}]
example_text

[{'role': 'assistant',
  'content': "😂 Ah, no I haven't! I'm dying to know, what's the meme about? Is it a funny cat or a ridiculous situation? Spill the beans! 🤣"}]

In [ ]:
tokenizer.apply_chat_template(example_text,tokenize=False)

"<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>assistant\n😂 Ah, no I haven't! I'm dying to know, what's the meme about? Is it a funny cat or a ridiculous situation? Spill the beans! 🤣<|im_end|>\n"

In [ ]:
def apply_chat_temp(example):

    example['prompt'] =[{'role':'user','content':example['prompt']}]

    return example


new_dataset = dataset.map(apply_chat_temp)
new_dataset[0]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

{'prompt': [{'content': 'Oh, I just saw the best meme - have you seen it?',
   'role': 'user'}],
 'chosen': "😂 Ah, no I haven't! I'm dying to know, what's the meme about? Is it a funny cat or a ridiculous situation? Spill the beans! 🤣",
 'rejected': "I'm an artificial intelligence language model, I don't have personal experiences or opinions. However, I can provide you with information on highly-rated and critically acclaimed films, as well as recommendations based on specific genres or themes. Would you like me to suggest some notable movies or discuss a particular genre of interest?"}

In [ ]:
new_templete = """<|im_start|>assistant\n{model_answer}<|im_end|>\n"""


def format_dataset(example):

    example['prompt'] = tokenizer.apply_chat_template(example['prompt'],tokenize=False)
    example['chosen'] = new_templete.format(model_answer =example['chosen'])
    example['rejected'] = new_templete.format(model_answer =example['rejected'])

    return example


formatted_dataset = new_dataset.map(format_dataset)
formatted_dataset[0]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

{'prompt': '<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nOh, I just saw the best meme - have you seen it?<|im_end|>\n',
 'chosen': "<|im_start|>assistant\n😂 Ah, no I haven't! I'm dying to know, what's the meme about? Is it a funny cat or a ridiculous situation? Spill the beans! 🤣<|im_end|>\n",
 'rejected': "<|im_start|>assistant\nI'm an artificial intelligence language model, I don't have personal experiences or opinions. However, I can provide you with information on highly-rated and critically acclaimed films, as well as recommendations based on specific genres or themes. Would you like me to suggest some notable movies or discuss a particular genre of interest?<|im_end|>\n"}

In [ ]:
model_8bit

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear8bitLt(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear8bitLt(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear8bitLt(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear8bitLt(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear8bitLt(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear8bitLt(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear8bitLt(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNor

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config= LoraConfig(

                        r = 32,
                        lora_alpha = 32,
                        target_modules =["q_proj","k_proj","v_proj","o_proj",
                  "gate_proj","up_proj","down_proj"],
                        lora_dropout = 0.05,
                        bias = "none",
                        task_type = "CAUSAL_LM"
)
lora_model = get_peft_model(model_8bit,lora_config)
lora_model.print_trainable_parameters()

trainable params: 36,929,536 || all params: 1,580,643,840 || trainable%: 2.3364


In [ ]:
lora_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=1536, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): 

In [ ]:
from trl import DPOTrainer, DPOConfig

dpo_config = DPOConfig(

                      output_dir = "./results",
                      per_device_train_batch_size = 2,
                      max_steps = 40,
                      gradient_accumulation_steps = 4,
                      learning_rate = 3e-5,
                      lr_scheduler_type ="cosine",
                      optim = "adamw_torch",
                      logging_steps = 10,
                      beta = 0.01,
                      report_to = "none"

)

In [ ]:
trainer = DPOTrainer(

                     model = lora_model,
                    # If no `ref_model` is provided, the trainer creates a copy of the current model (without LoRA adapters).,
                     train_dataset = formatted_dataset,
                    #  tokenizer = tokenizer,  old one
                    processing_class = tokenizer, # new one
                     args = dpo_config
)

trainer.train()

<ipython-input-14-ea251014c22d>:1: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `DPOTrainer.__init__`. Use `processing_class` instead.
  trainer = DPOTrainer(


Extracting prompt in train dataset:   0%|          | 0/300 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/300 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/300 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,0.541000
20,0.141800
30,0.016600
40,0.010600


TrainOutput(global_step=40, training_loss=0.1774996966123581, metrics={'train_runtime': 199.7703, 'train_samples_per_second': 1.602, 'train_steps_per_second': 0.2, 'total_flos': 0.0, 'train_loss': 0.1774996966123581, 'epoch': 1.0533333333333332})

In [ ]:
lora_path = "./lora_adapters"
lora_model.save_pretrained(lora_path)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
lora_path = "./lora_adapters"


name_name = "Qwen/Qwen2.5-1.5B-Instruct"




base_model = AutoModelForCausalLM.from_pretrained(

                                                  name_name,

                                                  trust_remote_code=True,
                                                  device_map="auto",
                                                  )

tokenizer = AutoTokenizer.from_pretrained(name_name, trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [ ]:
from peft import PeftModel

final_model = PeftModel.from_pretrained(base_model,lora_path)
final_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=1536, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(

In [ ]:
final_model = final_model.merge_and_unload()
final_model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotary_emb): Qw

In [ ]:
final_model.push_to_hub("Cagatayd/final_model")
tokenizer.push_to_hub("Cagatayd/final_model")